In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm
%run data_frames.ipynb

                  montos     inp        pib       fbc      roe      roa
fecha                                                                  
2000-04-01  7.911855e+08  191.94  100329.48  30688.07  0.01105 -0.01275
2000-07-01  2.243034e+09  198.11  101038.64  29361.68  0.01760 -0.01170
2000-10-01  7.573976e+08  204.14  109422.18  27006.68  0.02415 -0.01065
2001-01-01  6.559477e+08  209.08  101475.53  31964.81  0.03190 -0.00120
2001-04-01  1.236151e+08  215.71  104079.62  33305.56  0.03230 -0.00045


C:\Users\jimen\AppData\Local\Temp\ipykernel_13580\1236089611.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['fecha'] = pd.to_datetime(df['ano'].astype(str) + 'Q' + df['trimestre'].astype(str))


In [8]:
#Creación de rezagos
roe = final_df.copy()
max_lag = 4

for lag in range(1, max_lag + 1):
    roe[f'roa_lag_{lag}'] = roe['roa_diff2'].shift(lag)
    roe[f'pib_lag_{lag}'] = roe['pib_diff2'].shift(lag)
    roe[f'fbc_lag_{lag}'] = roe['fbc_diff2'].shift(lag)
    roe[f'montos_lag_{lag}'] = roe['montos_diff2'].shift(lag)
    roe[f'inp_lag_{lag}'] = roe['inp_diff2'].shift(lag)

roe.dropna(inplace=True)

#Definición de variables dependientes e independientes
Y = roe['roe_diff2'] 
X = roe[[col for col in roe.columns if 'lag' in col]]
X = add_constant(X)  # Se añade una constante al modelo

# Estimación del modelo
model = OLS(Y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              roe_diff2   R-squared:                       0.663
Model:                            OLS   Adj. R-squared:                  0.534
Method:                 Least Squares   F-statistic:                     5.124
Date:                Sun, 09 Jun 2024   Prob (F-statistic):           1.07e-06
Time:                        16:57:13   Log-Likelihood:                 171.39
No. Observations:                  73   AIC:                            -300.8
Df Residuals:                      52   BIC:                            -252.7
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.0014      0.003     -0.411   

In [11]:
roa = final_df.copy()
max_lag = 4

for lag in range(1, max_lag + 1):
    roa[f'roe_lag_{lag}'] = roa['roe_diff2'].shift(lag)
    roa[f'pib_lag_{lag}'] = roa['pib_diff2'].shift(lag)
    roa[f'fbc_lag_{lag}'] = roa['fbc_diff2'].shift(lag)
    roa[f'montos_lag_{lag}'] = roa['montos_diff2'].shift(lag)
    roa[f'inp_lag_{lag}'] = roa['inp_diff2'].shift(lag)

roa.dropna(inplace=True)

Y = roa['roa_diff2'] 
X = roa[[col for col in roa.columns if 'lag' in col]]
X = add_constant(X)  # Añade una constante al modelo

# Estimación del modelo
model = OLS(Y, X)
results = model.fit()

# Muestra los resultados
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              roa_diff2   R-squared:                       0.301
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     1.119
Date:                Sun, 09 Jun 2024   Prob (F-statistic):              0.360
Time:                        17:00:00   Log-Likelihood:                 202.28
No. Observations:                  73   AIC:                            -362.6
Df Residuals:                      52   BIC:                            -314.5
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.0007      0.002     -0.328   

MODELO DE CORRECCIÓN DE ERRORES

 Prueba de correlación Johansen

In [6]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

cols_for_johansen = ['roe', 'roa', 'pib', 'fbc', 'montos', 'inp']
df_coint = df[cols_for_johansen].dropna()
johansen_test = coint_johansen(df_coint, det_order=0, k_ar_diff=1)

print(johansen_test.trace_stat)
print(johansen_test.cvt)  # Valores críticos para el estadístico de traza
print(johansen_test.max_eig_stat)
print(johansen_test.cvm)  # Valores críticos para el estadístico del mayor eigenvalor

[200.09044183 103.06510989  51.80135031  22.99690789  10.67452854
   2.36938059]
[[ 91.109   95.7542 104.9637]
 [ 65.8202  69.8189  77.8202]
 [ 44.4929  47.8545  54.6815]
 [ 27.0669  29.7961  35.4628]
 [ 13.4294  15.4943  19.9349]
 [  2.7055   3.8415   6.6349]]
[97.02533193 51.26375959 28.80444241 12.32237936  8.30514795  2.36938059]
[[37.2786 40.0763 45.8662]
 [31.2379 33.8777 39.3693]
 [25.1236 27.5858 32.7172]
 [18.8928 21.1314 25.865 ]
 [12.2971 14.2639 18.52  ]
 [ 2.7055  3.8415  6.6349]]


Incluimos el error

In [7]:
vec_coint = johansen_test.evec[:, 0]
# Construir el término de corrección de error (toma el producto punto entre el vector de cointegración y las variables)
final_df['error_correction_term'] = df[cols_for_johansen].dot(vec_coint)

C:\Users\jimen\AppData\Local\Temp\ipykernel_30040\2960450233.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['error_correction_term'] = df[cols_for_johansen].dot(vec_coint)


In [8]:
final_df.head()

,montos_diff2,pib_diff2,fbc_diff2,roe_diff2,roa_diff2,inp_diff2,error_correction_term
fecha,,,,,,,
2000-10-01,-2.937485e+09,7674.38,-1028.61,0.000000e+00,1.734723e-18,-0.14,0.214722
2001-01-01,1.384186e+09,-16330.19,7313.13,1.200000e-03,8.400000e-03,-1.09,0.197719
2001-04-01,-4.308826e+08,10550.74,-3617.38,-7.350000e-03,-8.700000e-03,1.69,0.199794
2001-07-01,8.157196e+08,55.82,1669.80,-6.938894e-18,1.084202e-19,0.97,0.205132
2001-10-01,1.536507e+09,2359.69,-9699.97,0.000000e+00,0.000000e+00,-1.37,0.207742


Modelo de Corrección de Error (ECM) || ROA lin-lin segundas diferencias

In [9]:
Y = final_df['roa_diff2'] 

# Las variables independientes son las segundas diferencias de las variables y se  incluye el término de corrección de error
X = final_df[['roe_diff2', 'pib_diff2', 'fbc_diff2', 'error_correction_term', 'montos_diff2', 'inp_diff2']]  
X = add_constant(X)  # Se añade una constante al modelo

# Estimación del modelo
model = OLS(Y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              roa_diff2   R-squared:                       0.187
Model:                            OLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                     2.677
Date:                Sun, 09 Jun 2024   Prob (F-statistic):             0.0214
Time:                        00:53:54   Log-Likelihood:                 209.36
No. Observations:                  77   AIC:                            -404.7
Df Residuals:                      70   BIC:                            -388.3
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.00

Modelo de Corrección de Error (ECM) || ROE lin-lin segundas diferencias

In [10]:
Y = final_df['roe_diff2'] 
X = final_df[['roa_diff2', 'pib_diff2', 'fbc_diff2', 'error_correction_term', 'montos_diff2', 'inp_diff2']]
X = add_constant(X)
model = OLS(Y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              roe_diff2   R-squared:                       0.531
Model:                            OLS   Adj. R-squared:                  0.491
Method:                 Least Squares   F-statistic:                     13.21
Date:                Sun, 09 Jun 2024   Prob (F-statistic):           6.07e-10
Time:                        00:53:54   Log-Likelihood:                 170.06
No. Observations:                  77   AIC:                            -326.1
Df Residuals:                      70   BIC:                            -309.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.00

Optimizando el modelo de Rezagos Distribuidos

In [11]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Asegurarse de que final_df no contenga valores NaN
final_df = final_df.dropna()

# Se prepara el DataFrame para el análisis VIF
# Incluyendo todas las variables diferenciadas de segundo orden
vif_data = pd.DataFrame()
vif_data["Variable"] = final_df.columns

# Se Calcula VIF para cada variable en final_df
vif_data["VIF"] = [variance_inflation_factor(final_df.values, i) for i in range(final_df.shape[1])]

print(vif_data)

                Variable        VIF
0           montos_diff2   1.101868
1              pib_diff2   2.418189
2              fbc_diff2   1.444714
3              roe_diff2   2.132786
4              roa_diff2   1.229526
5              inp_diff2  20.748930
6  error_correction_term  20.811965


ROE

In [13]:

# Número de rezagos
n_lags = 4

# Crear rezagos para las variables independientes
for lag in range(1, n_lags + 1):
    final_df[[f'pib_diff2_lag{lag}', f'fbc_diff2_lag{lag}', f'montos_diff2_lag{lag}', f'inp_diff2_lag{lag}']] = final_df[['pib_diff2', 'fbc_diff2', 'montos_diff2', 'inp_diff2']].shift(lag)

# Eliminar las filas con valores NaN generados por los rezagos
final_df_lagged = final_df.dropna()

# Definir la variable dependiente y las independientes para ROE
Y_roe = final_df_lagged['roe_diff2']
X_roe = final_df_lagged.drop(['roe_diff2', 'roa_diff2'], axis=1)

# Agregar una constante a las variables independientes
X_roe = sm.add_constant(X_roe)

# Construir y ajustar el modelo
model_roe = sm.OLS(Y_roe, X_roe).fit()
print(model_roe.summary())


                            OLS Regression Results                            
Dep. Variable:              roe_diff2   R-squared:                       0.672
Model:                            OLS   Adj. R-squared:                  0.537
Method:                 Least Squares   F-statistic:                     4.974
Date:                Sun, 09 Jun 2024   Prob (F-statistic):           1.45e-06
Time:                        00:57:35   Log-Likelihood:                 172.33
No. Observations:                  73   AIC:                            -300.7
Df Residuals:                      51   BIC:                            -250.3
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.00

ROA

In [14]:
# Definir la variable dependiente y las independientes para ROA
Y_roa = final_df_lagged['roa_diff2']
X_roa = final_df_lagged.drop(['roe_diff2', 'roa_diff2'], axis=1)

# Agregar una constante a las variables independientes
X_roa = sm.add_constant(X_roa)

# Construir y ajustar el modelo
model_roa = sm.OLS(Y_roa, X_roa).fit()

# Mostrar el resumen del modelo
print(model_roa.summary())

                            OLS Regression Results                            
Dep. Variable:              roa_diff2   R-squared:                       0.331
Model:                            OLS   Adj. R-squared:                  0.055
Method:                 Least Squares   F-statistic:                     1.201
Date:                Sun, 09 Jun 2024   Prob (F-statistic):              0.290
Time:                        00:59:18   Log-Likelihood:                 203.89
No. Observations:                  73   AIC:                            -363.8
Df Residuals:                      51   BIC:                            -313.4
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.00

In [15]:
correlation_matrix = X_roe.corr()
print("Matriz de Correlación:")
print(correlation_matrix)

Matriz de Correlación:
                       const  montos_diff2  pib_diff2  fbc_diff2  inp_diff2  \
const                    NaN           NaN        NaN        NaN        NaN   
montos_diff2             NaN      1.000000   0.309906   0.115552  -0.000291   
pib_diff2                NaN      0.309906   1.000000   0.445293   0.010121   
fbc_diff2                NaN      0.115552   0.445293   1.000000  -0.024881   
inp_diff2                NaN     -0.000291   0.010121  -0.024881   1.000000   
error_correction_term    NaN     -0.004796  -0.023124   0.033014  -0.974625   
pib_diff2_lag1           NaN     -0.214880  -0.639042  -0.330871   0.030106   
fbc_diff2_lag1           NaN     -0.154124  -0.130746  -0.469992  -0.053483   
montos_diff2_lag1        NaN     -0.653802  -0.252243  -0.191400   0.000860   
inp_diff2_lag1           NaN      0.015201   0.044027  -0.028244   0.873630   
pib_diff2_lag2           NaN      0.169769   0.259874   0.011780   0.078733   
fbc_diff2_lag2           NaN 

In [16]:
vif = pd.DataFrame()
vif["Variable"] = X_roe.columns
vif["VIF"] = [variance_inflation_factor(X_roe.values, i) for i in range(X_roe.shape[1])]
print("Factor de Inflación de la Varianza (VIF):")
print(vif)

Factor de Inflación de la Varianza (VIF):
                 Variable           VIF
0                   const      7.323817
1            montos_diff2      4.709994
2               pib_diff2     14.791878
3               fbc_diff2      7.633513
4               inp_diff2  15647.701374
5   error_correction_term  63560.885689
6          pib_diff2_lag1     33.308321
7          fbc_diff2_lag1     23.020758
8       montos_diff2_lag1      8.584067
9          inp_diff2_lag1   2133.504130
10         pib_diff2_lag2     39.686046
11         fbc_diff2_lag2     34.323569
12      montos_diff2_lag2     10.366836
13         inp_diff2_lag2   2939.640427
14         pib_diff2_lag3     36.596119
15         fbc_diff2_lag3     23.972970
16      montos_diff2_lag3      8.497775
17         inp_diff2_lag3   2299.625195
18         pib_diff2_lag4     14.197855
19         fbc_diff2_lag4      7.889423
20      montos_diff2_lag4      4.690131
21         inp_diff2_lag4    294.688250


Mejorando el modelo (se quita multicolinialidad) y se eliminan variables no significativas

In [18]:
import statsmodels.api as sm

# Eliminación de 'inp_diff2' y sus rezagos
variables_a_eliminar = ['inp_diff2', 'inp_diff2_lag1', 'inp_diff2_lag2', 'inp_diff2_lag3', 'inp_diff2_lag4']
final_df_lagged = final_df_lagged.drop(columns=variables_a_eliminar)

# Inicializar variables para el proceso de eliminación hacia atrás
variables_significativas = final_df_lagged.columns.drop(['roe_diff2', 'roa_diff2'])
p_value_maximo = 1
umbral_significancia = 0.05  

# Proceso de eliminación hacia atrás
while p_value_maximo > umbral_significancia:
    # Se ajusta el modelo
    X_roe = sm.add_constant(final_df_lagged[variables_significativas])
    Y_roe = final_df_lagged['roe_diff2']
    modelo = sm.OLS(Y_roe, X_roe).fit()

    # Obtener el p-value máximo
    p_value_maximo = max(modelo.pvalues)
    variable_con_p_value_maximo = modelo.pvalues.idxmax()

    # Eliminar la variable menos significativa (excluyendo 'const')
    if p_value_maximo > umbral_significancia and variable_con_p_value_maximo != 'const':
        variables_significativas = variables_significativas.drop(variable_con_p_value_maximo)
    else:
        break  # Salir del bucle si no se puede eliminar más variables

# Ajustar y mostrar el modelo final
modelo_final = sm.OLS(Y_roe, sm.add_constant(final_df_lagged[variables_significativas])).fit()
print(modelo_final.summary())


                            OLS Regression Results                            
Dep. Variable:              roe_diff2   R-squared:                       0.597
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                     7.413
Date:                Sun, 09 Jun 2024   Prob (F-statistic):           3.92e-08
Time:                        01:00:07   Log-Likelihood:                 164.84
No. Observations:                  73   AIC:                            -303.7
Df Residuals:                      60   BIC:                            -273.9
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.0012      0.00